In [34]:
import sys
sys.path.append('..')
from util import get_table_by_id_fred
import pandas as pd
import numpy as np

# path = '../data/fred/'
# id = 'M2SL'
# data = pd.read_csv(path + id + '.csv', index_col=0, parse_dates=True)

In [35]:
path = '../data/fred'
start = '1950-01-01'
end = None
id = 'M2SL'
df_new, columns = get_table_by_id_fred(id, path, 6, start=start, end=end, if_log=True)

In [36]:
df_new['log_predict'] = df_new['log_M2SL'].shift(-6) - df_new['log_M2SL']
df_new = df_new.dropna()

In [37]:
df_new

,M2SL,log_M2SL,log_M2SL_diff_1,log_M2SL_diff_2,log_M2SL_diff_4,log_M2SL_diff_8,log_M2SL_diff_16,log_M2SL_diff_32,log_predict
DATE,,,,,,,,,
1961-09-01,329.5,5.797576,0.005783,0.011907,0.022404,0.047865,0.090798,0.139489,0.040446
1961-10-01,331.1,5.802420,0.004844,0.010627,0.020751,0.045097,0.091001,0.140502,0.042572
1961-11-01,333.4,5.809343,0.006923,0.011767,0.023673,0.046349,0.091986,0.142224,0.041422
1961-12-01,335.5,5.815622,0.006279,0.013202,0.023829,0.047614,0.089100,0.145396,0.040309
1962-01-01,337.5,5.821566,0.005944,0.012223,0.023989,0.046393,0.090168,0.144127,0.038651
...,...,...,...,...,...,...,...,...,...
2023-06-01,20820.3,9.943684,-0.000826,0.003633,-0.015735,-0.030798,-0.037468,0.105081,-0.003189
2023-07-01,20793.2,9.942381,-0.001302,-0.002128,-0.005252,-0.029003,-0.043658,0.092331,-0.001877
2023-08-01,20765.3,9.941039,-0.001343,-0.002645,0.000988,-0.025856,-0.045051,0.083731,-0.000805


In [12]:
# Split into training and testing sets (for instance, use 80% of the data for training and 20% for testing)
train_size = int(len(df_new) * 0.8)
train, test = df_new[:train_size], df_new[train_size:]

X_train, y_train = train[columns], train['log_predict']
X_test, y_test = test[columns], test['log_predict']

In [29]:
from sklearn.pipeline import Pipeline

from safeRegressors import SafeSVR
from util import TIMEOUT
from sklearn.preprocessing import StandardScaler
import logging
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from safeRegressors import SafeRandomForestRegressor, SafeSVR, TimeoutException
import pickle

logger = logging.getLogger('training')

def objective_svm(trial, df_train_X, df_train_y):
  # Define the hyperparameter configuration space
  k = trial.suggest_int('k', 5, len(df_train_X.columns))
  C = trial.suggest_float('C', 1e-3, 1e2,log=True)
  epsilon = trial.suggest_float('epsilon', 1e-3, 1e1, log=True)
  kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
  gamma = trial.suggest_float('gamma', 1e-4, 1e1, log=True)

  # Model setup
  model = SafeSVR(C=C,  kernel=kernel, gamma=gamma, epsilon=epsilon, timeout=TIMEOUT)

  pipeline = Pipeline([
      ('scaler', StandardScaler()),  # Add scaler here
      ('svr', model),
  ])

  total_mses = 0
  try:
      X_train = df_train_X.copy().values
      y_train = df_train_y.copy().values.ravel()

      predictions = cross_val_predict(pipeline, X_train, y_train, cv=5, n_jobs=-1)

      mse = mean_squared_error(y_train, predictions)
      total_mses += mse
    
  except TimeoutException:
      logger.error("A timeout has occurred during model fitting.")
      # Return a large MSE value to penalize this result
      return float('inf')

In [31]:
from sklearn.feature_selection import SelectKBest, f_regression

def objective_random_forest(trial, df_train_X, df_train_y):
  # Define the hyperparameter configuration space
  k = trial.suggest_int('k', 5, len(df_train_X.columns))
  n_estimators = trial.suggest_int('n_estimators', 20, 160)
  max_depth = trial.suggest_int('max_depth', 10, 50)
  min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
  max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
  bootstrap = trial.suggest_categorical('bootstrap', [True, False])
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 6, 200)


  # Model setup
  model = SafeRandomForestRegressor(
      n_estimators=n_estimators,
      max_depth=max_depth,
      min_samples_split=min_samples_split,
      min_samples_leaf=min_samples_leaf,
      max_features=max_features,
      bootstrap=bootstrap,
      max_leaf_nodes=max_leaf_nodes,
      timeout=TIMEOUT
  )

  pipeline = Pipeline([
      ('truncate', SelectKBest(f_regression, k=k)), # Adjust 'k' as needed
      ('regress', model),
  ])


  total_mses = 0
  try:
      df_train_X = df_train_X
      df_train_y = df_train_y

      X_train = df_train_X.copy().values
      y_train = df_train_y.copy().values.ravel()

      predictions = cross_val_predict(pipeline, X_train, y_train, cv=5, n_jobs=5)
      mse = mean_squared_error(y_train, predictions)
    
      return mse
  except TimeoutException:
      logger.error("A timeout has occurred during model fitting.")
      # Return a large MSE value to penalize this result
      return float('inf')

In [32]:
import optuna
study_rf = optuna.create_study()  

study_rf.optimize(lambda trial: objective_random_forest(trial, X_train, y_train), n_trials=10)

[I 2024-06-05 14:46:10,269] A new study created in memory with name: no-name-6c474aaa-f364-45ff-a40f-865606cd4fe2
[I 2024-06-05 14:46:11,864] Trial 0 finished with value: 0.00023401782599036326 and parameters: {'k': 6, 'n_estimators': 155, 'max_depth': 38, 'min_samples_split': 11, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False, 'max_leaf_nodes': 28}. Best is trial 0 with value: 0.00023401782599036326.
[I 2024-06-05 14:46:12,126] Trial 1 finished with value: 0.00024289211249076042 and parameters: {'k': 6, 'n_estimators': 126, 'max_depth': 34, 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False, 'max_leaf_nodes': 188}. Best is trial 0 with value: 0.00023401782599036326.
[I 2024-06-05 14:46:12,427] Trial 2 finished with value: 0.00023695861917425007 and parameters: {'k': 5, 'n_estimators': 105, 'max_depth': 44, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': None, 'bootstrap': True, 'max_leaf_nodes': 49}. Best is tri

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
# Features and predictors

best_params = study_rf.best_params
best_pipeline = Pipeline([
        ('truncate', SelectKBest(f_regression, k=best_params['k'])), # Adjust 'k' as needed
        ('regress', SafeRandomForestRegressor(
          n_estimators=best_params['n_estimators'],
          max_depth=best_params['max_depth'],
          min_samples_split=best_params['min_samples_split'],
          min_samples_leaf=best_params['min_samples_leaf'],
          max_features=best_params['max_features'],
          bootstrap=best_params['bootstrap'],
          max_leaf_nodes=best_params['max_leaf_nodes'],
          timeout=TIMEOUT
    ))])


best_pipeline.fit(X_train, y_train)

pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.0009163611831543184


In [25]:
pred

array([0.03823168, 0.0424143 , 0.05293143, 0.0536318 , 0.05056892,
       0.03562755, 0.03388911, 0.02852802, 0.03885493, 0.03450499,
       0.03735552, 0.03595383, 0.03712398, 0.04405474, 0.04005191,
       0.0422788 , 0.03965177, 0.04208199, 0.03675399, 0.04584539,
       0.02841463, 0.03054342, 0.04106279, 0.02304307, 0.01980032,
       0.03341488, 0.03214727, 0.03790275, 0.0457419 , 0.03505687,
       0.029483  , 0.03869284, 0.03913402, 0.04000614, 0.03902794,
       0.03808089, 0.03852215, 0.03668573, 0.04095354, 0.03407089,
       0.03252811, 0.03747572, 0.0322514 , 0.04388778, 0.03791869,
       0.04009463, 0.03306259, 0.03790133, 0.03070782, 0.0206293 ,
       0.02880428, 0.03437381, 0.03716899, 0.02733478, 0.04647975,
       0.04379359, 0.03266177, 0.03465627, 0.03976514, 0.03897236,
       0.04164347, 0.04132917, 0.04211976, 0.04355917, 0.04124411,
       0.04099564, 0.04170007, 0.03641296, 0.03465434, 0.03401142,
       0.03174363, 0.03496319, 0.03621989, 0.03255691, 0.03288

In [24]:
train['log_predict']

DATE
1961-09-01    0.040446
1961-10-01    0.042572
1961-11-01    0.041422
1961-12-01    0.040309
1962-01-01    0.038651
                ...   
2010-12-01    0.038907
2011-01-01    0.054424
2011-02-01    0.067513
2011-03-01    0.063339
2011-04-01    0.060038
Name: log_predict, Length: 596, dtype: float64

In [20]:
# save the model
import pickle

with open('../models/linear_regression_m2sl.pkl', 'wb') as f:
    pickle.dump(model, f)


In [18]:

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
# Features and predictors

model = SVR()
model.fit(X_train, y_train)

pred = model.predict(X_test)
mse = mean_squared_error(y_test, pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.0011537656852879561


In [16]:
y_pred_naive = X_test['log_M2SL_diff_4']
mse_naive = mean_squared_error(y_test, y_pred_naive)
print(f'Mean Squared Error (Naive): {mse_naive}')

Mean Squared Error (Naive): 0.0010814485939329072


In [6]:
import numpy as np

period = 6
df['log_predict'] = np.log(df[id].shift(-period) / df[id])

,M2SL,log_M2SL,log_M2SL_diff_1,log_M2SL_diff_2,log_M2SL_diff_4,log_M2SL_diff_8,log_M2SL_diff_16,log_M2SL_diff_32,log_predict
DATE,,,,,,,,,
1959-01-01,286.6,5.658088,NaN,NaN,NaN,NaN,NaN,NaN,0.029566
1959-02-01,287.7,5.661918,0.003831,NaN,NaN,NaN,NaN,NaN,0.029792
1959-03-01,289.2,5.667118,0.005200,0.009031,NaN,NaN,NaN,NaN,0.025603
1959-04-01,290.1,5.670226,0.003107,0.008307,NaN,NaN,NaN,NaN,0.021822
1959-05-01,292.2,5.677438,0.007213,0.010320,0.019351,NaN,NaN,NaN,0.016630
...,...,...,...,...,...,...,...,...,...
2023-12-01,20754.0,9.940494,0.002354,0.003006,-0.000544,0.000443,-0.040829,0.030771,NaN
2024-01-01,20754.2,9.940504,0.000010,0.002364,0.002089,-0.004006,-0.037121,0.015774,NaN
2024-02-01,20748.6,9.940234,-0.000270,-0.000260,0.002746,-0.003450,-0.034248,0.013367,NaN


In [8]:
from util import remove_nan


df, _ = remove_nan(df, type='top')

df, _ = remove_nan(df, type='bottom')


In [9]:
df

,M2SL,log_M2SL,log_M2SL_diff_1,log_M2SL_diff_2,log_M2SL_diff_4,log_M2SL_diff_8,log_M2SL_diff_16,log_M2SL_diff_32,log_predict
DATE,,,,,,,,,
1961-09-01,329.5,5.797576,0.005783,0.011907,0.022404,0.047865,0.090798,0.139489,0.040446
1961-10-01,331.1,5.802420,0.004844,0.010627,0.020751,0.045097,0.091001,0.140502,0.042572
1961-11-01,333.4,5.809343,0.006923,0.011767,0.023673,0.046349,0.091986,0.142224,0.041422
1961-12-01,335.5,5.815622,0.006279,0.013202,0.023829,0.047614,0.089100,0.145396,0.040309
1962-01-01,337.5,5.821566,0.005944,0.012223,0.023989,0.046393,0.090168,0.144127,0.038651
...,...,...,...,...,...,...,...,...,...
2023-05-01,20837.5,9.944510,0.004459,-0.003124,-0.018317,-0.033116,-0.034039,0.114055,-0.006369
2023-06-01,20820.3,9.943684,-0.000826,0.003633,-0.015735,-0.030798,-0.037468,0.105081,-0.003189
2023-07-01,20793.2,9.942381,-0.001302,-0.002128,-0.005252,-0.029003,-0.043658,0.092331,-0.001877


In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, shuffle=False)

# shuffle the train data
train = train.sample(frac=1).reset_index(drop=True)


df_train_y = train['log_predict']
df_train_X = train.drop(columns=['log_predict'])

df_test_y = test['log_predict']
df_test_X = test.drop(columns=['log_predict'])



ImportError: cannot import name 'split_train_test' from 'util' (/home/ken/git/portfolio-2/notebooks/../util.py)